[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/birds-eye.ipynb)

# Project flow

LaminDB enables to model data flow on the entire-project level.

Here, we walk through exemplified app uploads, pipelines & notebooks following [Schmidt _et al._, 2022](https://pubmed.ncbi.nlm.nih.gov/35113687/): A genome-wide CRISPR screen reading out a phenotypic endpoint on T cells is paired with scRNA-seq to generate insights into IFN-γ production.

These insights get linked back to the original data through the steps taken in the project to provide context for interpretation & future decision making.

:::{dropdown} More specifically: Why should I care about data flow?

Data flow tracks data sources & transformations to trace biological insights, verify experimental outcomes, meet regulatory standards, increase the robustness of research and optimize the feedback loop of team-wide learning iterations.

While tracking data flow is easier when it's governed by deterministic pipelines, it becomes hard when it's governed by interactive human-driven analyses.

LaminDB interfaces workflow mangers for the former and embraces the latter.

:::

## Setup

Init a test instance:

In [ ]:
!lamin init --storage ./mydata

Import lamindb:

In [ ]:
import lamindb as ln
from IPython.display import Image, display

## Steps

In the following, we walk through exemplified steps. The full notebooks are in [this repository](https://github.com/laminlabs/rnd-demo).

### App upload of phenotypic data

Register data through app upload from wetlab by `testuser1`:

In [ ]:
ln.setup.login("testuser1")
transform = ln.Transform(name="Upload GWS CRISPRa result", type="app")
ln.track(transform)
output_path = ln.dev.datasets.schmidt22_crispra_gws_IFNG(ln.settings.storage)
output_file = ln.File(output_path, description="Raw data of schmidt22 crispra GWS")
output_file.save()

### Hit identification in notebook

Access, transform & register data in drylab by `testuser2`:

In [ ]:
ln.setup.login("testuser2")
transform = ln.Transform(name="GWS CRIPSRa analysis", type="notebook")
ln.track(transform)
# access
input_file = ln.File.filter(key="schmidt22-crispra-gws-IFNG.csv").one()
# identify hits
input_df = input_file.load().set_index("id")
output_df = df[df["pos|fdr"] < 0.01].copy()
# register hits in output file
ln.File(output_df, description="hits from schmidt22 crispra GWS").save()

Inspect data flow:

In [ ]:
file = ln.File.filter(description="hits from schmidt22 crispra GWS").one()
file.view_lineage()

### Sequencer upload

Upload files from sequencer:

In [ ]:
ln.setup.login("testuser1")
ln.track(ln.Transform(name="Chromium 10x upload", type="pipeline"))
# register output files of upload
upload_dir = ln.dev.datasets.dir_scrnaseq_cellranger(
    "perturbseq", basedir=ln.settings.storage, output_only=False
)
ln.File(upload_dir.parent / "fastq/perturbseq_R1_001.fastq.gz").save()
ln.File(upload_dir.parent / "fastq/perturbseq_R2_001.fastq.gz").save()
ln.setup.login("testuser2")

### Track scRNA-seq bioinformatics pipeline

Process uploaded files using a script or workflow manager: {doc}`pipelines` and obtain 3 output files in a directory `filtered_feature_bc_matrix/`:

In [ ]:
transform = ln.Transform(name="Cell Ranger", version="7.2.0", type="pipeline")
ln.track(transform)
# access uploaded files as inputs for the pipeline
input_files = ln.File.filter(key__startswith="fastq/perturbseq").all()
input_paths = [file.stage() for file in files]
# register output files
output_files = ln.File.from_dir("./mydata/perturbseq/filtered_feature_bc_matrix/")
ln.save(output_files)

Inspect data lineage:

In [ ]:
output_files[0].view_lineage()

Post-process Cell Ranger:

In [ ]:
transform = ln.Transform(name="Postprocess Cell Ranger", version="2.0", type="pipeline")
ln.track(transform)
input_files = [f.stage() for f in output_files]
output_path = ln.dev.datasets.schmidt22_perturbseq(basedir=ln.settings.storage)
output_file = ln.File(output_path, description="perturbseq counts")
output_file.save()

### Integrate scRNA-seq & CRISPR screen results

Integrate data in a notebook:

In [ ]:
transform = ln.Transform(
    name="Perform single cell analysis, integrate with CRISPRa screen",
    type="notebook",
)
ln.track(transform)

file_ps = ln.File.filter(description__icontains="perturbseq").one()
adata = file_ps.load()
screen_hits = file_hits.load()

import scanpy as sc

sc.tl.score_genes(adata, adata.var_names.intersection(screen_hits.index).tolist())
filesuffix = "_fig1_score-wgs-hits.png"
sc.pl.umap(adata, color="score", show=False, save=filesuffix)
filepath = f"figures/umap{filesuffix}"
file = ln.File(filepath, key=filepath)
file.save()
filesuffix = "fig2_score-wgs-hits-per-cluster.png"
sc.pl.matrixplot(
    adata, groupby="cluster_name", var_names=["score"], show=False, save=filesuffix
)
filepath = f"figures/matrixplot_{filesuffix}"
file = ln.File(filepath, key=filepath)
file.save()

### Review results

Let's load one of the plots:

In [ ]:
ln.track()
file = ln.File.filter(key__contains="figures/matrixplot").one()
file.stage()
display(Image(filename=file.path))

We see that the image file is tracked as an input of the current notebook. The input is highlighted, the notebook follows at the bottom:

In [ ]:
file.view_lineage()

Alternatively, we can also look at the sequence of transforms:

In [ ]:
transform = ln.Transform.search("Bird's eye view", return_queryset=True).first()

In [ ]:
transform.parents.df()

In [ ]:
transform.view_parents()

## Understand runs

We tracked pipeline and notebook runs through {class}`~docs:lamindb.dev.run_context`, which stores a {class}`~docs:lamindb.Transform` and a {class}`~docs:lamindb.Run` record as a global context.

{class}`~lamindb.File` objects are the inputs and outputs of runs. 

:::{dropdown} What if I don't want a global context?

Sometimes, we don't want to create a global run context but manually pass a run when creating a file:
```python
run = ln.Run(transform=transform)
ln.File(filepath, run=run)
```

:::

:::{dropdown} When does a file appear as a run input?

When accessing a file via `stage()`, `load()` or `backed()`, two things happen:

1. The current run gets added to `file.input_of`
2. The transform of that file gets added as a parent of the current transform

You can then switch off auto-tracking of run inputs if you set `ln.settings.track_run_inputs = False`: {doc}`docs:faq/track-run-inputs`

You can also track run inputs on a case by case basis via `is_run_input=True`, e.g., here:
```python
file.load(is_run_input=True)
```

:::

## Query by provenance

We can query or search for the notebook that created the file:

In [ ]:
transform = ln.Transform.search("GWS CRIPSRa analysis", return_queryset=True).first()

And then find all the files created by that notebook:

In [ ]:
ln.File.filter(transform=transform).df()

Which transform ingested a given file?

In [ ]:
file = ln.File.filter().first()
file.transform

And which user?

In [ ]:
file.created_by

Which transforms were created by a given user?

In [ ]:
users = ln.User.lookup()

In [ ]:
ln.Transform.filter(created_by=users.testuser2).df()

Which notebooks were created by a given user?

In [ ]:
ln.Transform.filter(created_by=users.testuser2, type="notebook").df()

We can also view all recent additions to the entire database:

In [ ]:
ln.view()

In [ ]:
!lamin login testuser1
!lamin delete --force mydata
!rm -r ./mydata